In [1]:
%load_ext autoreload
%autoreload 2

import os

print("original dir: ", os.getcwd())

if os.getcwd().endswith("NewMethod"):
    new_path = "../"
    os.chdir(new_path)
    print("changed dir: ", os.getcwd())
    
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

original dir:  d:\我的\大学\3暑\创新实践\repo\Nonlinear-Erasure-Code\src\NewMethod
changed dir:  d:\我的\大学\3暑\创新实践\repo\Nonlinear-Erasure-Code\src
device:  cuda:0


In [2]:
import datetime

TASK_CONFIG = {
    "TASK": "MNIST",  # ARG
    "DATE": datetime.datetime.now().strftime("%Y_%m_%d"),
    "MODEL": "LeNet5",
}

读取数据集

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 设置数据转换
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

# 设置数据集（训练集与测试集合）

"""
MNIST:
image: (1, 28, 28), label: (0-9)

FashionMNIST:
image: (1, 28, 28), label: (0-9)

CIFAR10:
image: (3, 32, 32), label: (0-9)
"""

print(f"当前任务为 {TASK_CONFIG['TASK']}")

# ARG

train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# train_dataset = datasets.FashionMNIST(
#     root="./data", train=True, download=True, transform=transform
# )
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataset = datasets.FashionMNIST(
#     root="./data", train=False, download=True, transform=transform
# )
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Data is ready!")

当前任务为 MNIST
Data is ready!


设置部分参数

In [4]:
# ARG
K = 4
R = 2
N = K + R
original_data_shape = tuple(train_dataset[0][0].shape)
num_classes = 10
print(f"K: {K}")
print(f"R: {R}")
print(f"N: {N}")
print(f"data_shape: {original_data_shape}")
print(f"num_classes: {num_classes}")

K: 4
R: 2
N: 6
data_shape: (1, 28, 28)
num_classes: 10


定义 base model

In [5]:
import torch

from base_model.MyModel1 import MyModel1
from base_model.LeNet5 import LeNet5

# 引入 base model, 该model将在后续全部过程中使用
# ResNet
assert TASK_CONFIG["MODEL"] == "LeNet5"
model = LeNet5(input_dim=original_data_shape, num_classes=num_classes)

In [6]:
# 读取模型
base_model_path = (
    f"./base_model/{TASK_CONFIG['MODEL']}/{TASK_CONFIG['TASK']}/model.pth"
)
print(f"base_model_path: {base_model_path}")

model.load_state_dict(torch.load(base_model_path, map_location=device))
conv_segment = model.get_conv_segment()
fc_segment = model.get_fc_segment()
model.to(device)
model.eval()

print("Model is ready!")

base_model_path: ./base_model/LeNet5/MNIST/model.pth
Model is ready!


验证 base model 准确率

In [7]:
# 测试循环
model.eval()  # 设置模型为评估模式

correct = 0
total = 0
with torch.no_grad():  # 在评估过程中不计算梯度
    for data, target in train_loader:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"训练集-> 总量: {total}, 正确数量: {correct}, 准确率: {100 * correct / total}%")

correct = 0
total = 0
with torch.no_grad():  # 在评估过程中不计算梯度
    for data, target in test_loader:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"测试集-> 总量: {total}, 正确数量: {correct}, 准确率: {100 * correct / total}%")

训练集-> 总量: 60000, 正确数量: 59612, 准确率: 99.35333333333334%
测试集-> 总量: 10000, 正确数量: 9891, 准确率: 98.91%


验证 model 和 conv_segment, fc_segment 的输出是否一致

In [8]:
x = torch.randn(1, *original_data_shape).to(device)
model.to(device)
y = model(x)
print(y.data)

z = conv_segment(x)
z = z.flatten(1)
z = fc_segment(z)
print(z.data)
print(torch.allclose(y, z))

tensor([[-2.2468, -2.0310,  1.3496,  2.1254, -1.6824, -0.1339, -0.2846,  0.5037,
          5.6436, -2.7149]], device='cuda:0')
tensor([[-2.2468, -2.0310,  1.3496,  2.1254, -1.6824, -0.1339, -0.2846,  0.5037,
          5.6436, -2.7149]], device='cuda:0')
True


设置另一部分参数

In [9]:
from util.util import cal_input_shape


conv_output_shape = model.calculate_conv_output(input_dim=original_data_shape)
print(f"conv_output_shape: {conv_output_shape}")
assert conv_output_shape[2] % K == 0

split_conv_output_shape = (
    conv_output_shape[0],
    conv_output_shape[1],
    conv_output_shape[2] // K,
)
print(f"split_conv_output_shape: {split_conv_output_shape}")

conv_segment.to('cpu')
split_data_range = cal_input_shape(
    model=conv_segment,
    original_input_shape=original_data_shape,
    original_output_shape=conv_output_shape,
    split_num=K,
)
print(f"split_data_range: {split_data_range}")

# print(conv_segment)
print(
    f"split_conv_output_data_shape from split_data_shape: {[tuple(conv_segment(torch.randn(1, _[0], _[1], _[3] - _[2])).shape) for _ in split_data_range]}"
)

split_data_shapes = [
    (
        _[0],
        _[1],
        _[3] - _[2],
    )
    for _ in split_data_range
]
print(f"split_data_shapes: {split_data_shapes}")

split_data_shape = split_data_shapes[0]
print(f"choose the first one as the split_data_shape: {split_data_shape}")

conv_output_shape: (16, 4, 4)
split_conv_output_shape: (16, 4, 1)
split_data_range: [(1, 28, 0, 19), (1, 28, 4, 23), (1, 28, 8, 27), (1, 28, 12, 28)]
split_conv_output_data_shape from split_data_shape: [(1, 16, 4, 1), (1, 16, 4, 1), (1, 16, 4, 1), (1, 16, 4, 1)]
split_data_shapes: [(1, 28, 19), (1, 28, 19), (1, 28, 19), (1, 28, 16)]
choose the first one as the split_data_shape: (1, 28, 19)


验证分割后的输入，能够恰好恢复出原始输出

In [10]:
x = torch.randn(1, *original_data_shape).to(device)
conv_segment.to(device)
y = conv_segment(x)
print(f"y.shape: {y.shape}")

x_split = [x[:, :, :, _[2]:_[3]] for _ in split_data_range]
y_split = [conv_segment(_x) for _x in x_split]
print(f"y_split.shape: {[tuple(_y.shape) for _y in y_split]}")

y_hat = torch.cat(y_split, dim=3)
print(f"y_hat.shape: {y_hat.shape}")

# |A-B| <= atol + rtol * |B|
print(f"y和y_hat是否相等: {torch.allclose(y_hat, y, rtol=1e-08, atol=1e-05)}")

diff = torch.abs(y_hat - y)
epsilon = 0.0001
print(f"y和y_hat是否相等: {torch.all(diff <= epsilon)}")
# print(torch.allclose(y_split[0], y[:, :, :, 0:5]))
# print(torch.allclose(y_split[1], y[:, :, :, 5:10]))
# print(torch.allclose(y_split[2], y[:, :, :, 10:15]))
# print(torch.allclose(y_split[3], y[:, :, :, 15:20]))

# print(y[0][0][0] == y_hat[0][0][0])
# print(y[0][0][0])
# print(y_hat[0][0][0])
# y = x
# y_split = x_split
# for layer in conv_segment:
#     print(layer)
#     y = layer(y)
#     y_split = [layer(_y) for _y in y_split]
#     print(f"y.shape: {y.shape}")
#     print(f"y_split.shape: {[tuple(_.shape) for _ in y_split]}")
#     print(y[0][0][0])
#     print(y_split[0][0][0][0])
#     print(y_split[1][0][0][0])
#     print(y_split[2][0][0][0])
#     print(y_split[3][0][0][0])

y.shape: torch.Size([1, 16, 4, 4])
y_split.shape: [(1, 16, 4, 1), (1, 16, 4, 1), (1, 16, 4, 1), (1, 16, 4, 1)]
y_hat.shape: torch.Size([1, 16, 4, 4])
y和y_hat是否相等: True
y和y_hat是否相等: True


定义 Encoder Decoder

In [42]:
from encoder.mlp_encoder import MLPEncoder
from encoder.conv_encoder import CatChannelConvEncoder, CatBatchSizeConvEncoder
from decoder.mlp_decoder import MLPDecoder, ShrinkMLPDecoder
from decoder.conv_decoder import CatChannelConvDecoder, CatBatchSizeConvDecoder
from decoder.sparse_mlp_decoder import SparseMLPDecoder
from decoder.cnn_decoder import CNNDecoder

print(f"split_data_shape: {split_data_shape}")
print(f"split_conv_output_shape: {split_conv_output_shape}")

# ARG

# encoder = MLPEncoder(num_in=K, num_out=R, in_dim=split_data_shape)
encoder = CatChannelConvEncoder(num_in=K, num_out=R, in_dim=split_data_shape)

# decoder = MLPDecoder(num_in=N, num_out=K, in_dim=split_conv_output_shape)
decoder = ShrinkMLPDecoder(num_in=N, num_out=K, in_dim=split_conv_output_shape)
# decoder = CatChannelConvDecoder(num_in=N, num_out=K, in_dim=split_conv_output_shape)
# decoder = SparseMLPDecoder(num_in=N, num_out=K, in_dim=split_conv_output_shape)
# decoder = CNNDecoder(num_in=N, num_out=K, in_dim=split_conv_output_shape)

split_data_shape: (1, 28, 19)
split_conv_output_shape: (16, 4, 1)


In [ ]:
# print(torch.cuda.memory_summary())

In [43]:
def getModelSize(model):
    param_size = 0
    param_sum = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
        param_sum += param.nelement()
    buffer_size = 0
    buffer_sum = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
        buffer_sum += buffer.nelement()
    all_size = (param_size + buffer_size) / 1024 / 1024
    print("模型总大小为：{:.3f}MB".format(all_size))
    return (param_size, param_sum, buffer_size, buffer_sum, all_size)

getModelSize(encoder)
getModelSize(decoder)
print()

模型总大小为：1.112MB
模型总大小为：1.175MB



训练 Encoder Decoder

In [44]:
epoch_num = 5  # ARG
print(f"epoch_num: {epoch_num}")

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm

print(f"Train dataset: {len(train_dataset)}")
print("image size: ", train_dataset[0][0].size())

# 定义损失函数
criterion = nn.MSELoss()
criterion2 = nn.CrossEntropyLoss()
optimizer_encoder = optim.SGD(encoder.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)
optimizer_decoder = optim.SGD(decoder.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)

model.to(device)
conv_segment.to(device)
fc_segment.to(device)
encoder.to(device)
decoder.to(device)

model.eval()
conv_segment.eval()
fc_segment.eval()
encoder.train()
decoder.train()

model.eval()
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        m.track_running_stats = False

loss_list = [[] for _ in range(epoch_num)]

for epoch in range(epoch_num):
    train_loader_tqdm = tqdm(
        train_loader,
        desc=f"Epoch {epoch+1}/{epoch_num}",
        bar_format="{l_bar}{bar:20}{r_bar}",
    )
    correct = 0
    correct_truth = 0
    total = 0
    for images, labels in train_loader_tqdm:
        images = images.to(device)
        labels = labels.to(device)

        # split image tensor(64, 3, 32, 32) -> [tensor(64, 3, 32, 8) * K]
        images_list = []
        for _1, _2, start, end in split_data_range:
            images_list.append(images[:, :, :, start:end].clone())

        pad = (0, 3, 0, 0)
        images_list[-1] = F.pad(images_list[-1], pad, "constant", value=0)

        ground_truth = conv_segment(images)
        ground_truth = ground_truth.view(ground_truth.size(0), -1)

        # forward
        images_list += encoder(images_list)
        output_list = []
        for i in range(N):
            output = conv_segment(images_list[i])
            output_list.append(output)
        # losed_output_list = lose_something(output_list, self.lose_device_index)
        decoded_output_list = decoder(output_list)
        output = torch.cat(decoded_output_list, dim=3)
        output = output.view(output.size(0), -1)

        loss = criterion(output, ground_truth)
        # loss = criterion2(fc_segment(output), fc_segment(ground_truth))

        loss_list[epoch].append(loss.item())

        # backward
        optimizer_encoder.zero_grad()
        optimizer_decoder.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=10.0)
        # torch.nn.utils.clip_grad_norm_(decoder.parameters(), max_norm=10.0)
        optimizer_encoder.step()
        optimizer_decoder.step()

        # calculate accuracy
        _, predicted = torch.max(fc_segment(output).data, 1)
        _, predicted_truth = torch.max(fc_segment(ground_truth.data), 1)
        # print(predicted)
        # print(predicted_truth)
        # print(labels)
        correct += (predicted == labels).sum().item()
        correct_truth += (predicted_truth == labels).sum().item()
        total += labels.size(0)

        train_loader_tqdm.set_postfix(loss=loss.item())

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")
    print(f"Train Accuracy: {100 * correct / total}%")
    print(f"Original Accuracy: {100 * correct_truth / total}%")
    # 27%
    # 10%

epoch_num: 5
Train dataset: 60000
image size:  torch.Size([1, 28, 28])


Epoch 1/5: 100%|████████████████████| 938/938 [01:06<00:00, 14.16it/s, loss=1.06]


Epoch: 1, Loss: 1.063309669494629
Train Accuracy: 60.825%
Original Accuracy: 99.35333333333334%


Epoch 2/5: 100%|████████████████████| 938/938 [01:05<00:00, 14.31it/s, loss=0.775]


Epoch: 2, Loss: 0.7745357155799866
Train Accuracy: 94.06%
Original Accuracy: 99.35333333333334%


...

Epoch: 3, Loss: 0.7331961393356323
Train Accuracy: 96.62833333333333%
Original Accuracy: 99.35333333333334%


Epoch 4/5: 100%|████████████████████| 938/938 [01:05<00:00, 14.34it/s, loss=0.513]


Epoch: 4, Loss: 0.5130935907363892
Train Accuracy: 97.64333333333333%
Original Accuracy: 99.35333333333334%


Epoch 5/5: 100%|████████████████████| 938/938 [01:05<00:00, 14.33it/s, loss=0.499]

Epoch: 5, Loss: 0.49905845522880554
Train Accuracy: 98.05166666666666%
Original Accuracy: 99.35333333333334%


Evaluation

In [ ]:
def lose_something(output_list, lose_num):
    if lose_num == 0:
        return output_list
    
    lose_index = torch.randperm(len(output_list))[:lose_num]
    losed_output_list = []

    for i in range(len(output_list)):

        if i in lose_index:

            losed_output_list.append(torch.zeros_like(output_list[i]))
        else:

            losed_output_list.append(output_list[i])
    return losed_output_list

In [ ]:
import torch

from tqdm import tqdm

from dataset.image_dataset import ImageDataset
from util.split_data import split_vector

conv_segment.to(device)
fc_segment.to(device)
model.to(device)
encoder.to(device)
decoder.to(device)

conv_segment.eval()
fc_segment.eval()
model.eval()
encoder.eval()
decoder.eval()

def evaluation(loader, loss_num):
    original_correct = 0
    merge_correct = 0
    correct = 0
    total = 0
    with torch.no_grad():
        loader_tqdm = tqdm(
            loader,
            desc=f"Evaluating...",
            bar_format="{l_bar}{bar:20}{r_bar}",
        )
        for images, labels in loader_tqdm:
            images = images.to(device)
            labels = labels.to(device)
            
            # split image tensor(64, 3, 32, 32) -> [tensor(64, 3, 32, 8) * K]
            images_list = []
            for _1, _2, start, end in split_data_range:
                images_list.append(images[:, :, :, start:end].clone())
        
            pad = (0, 3, 0, 0)
            images_list[-1] = F.pad(images_list[-1], pad, "constant", value=0)

            _, predicted = torch.max(model(images).data, 1)
            original_correct += (predicted == labels).sum().item()

            output = conv_segment(images)
            output = output.view(output.size(0), -1)
            output = fc_segment(output)
            _, predicted = torch.max(output.data, 1)
            merge_correct += (predicted == labels).sum().item()

            imageDataset_list = [
                ImageDataset(images) for images in images_list + encoder(images_list)
            ]
            output_list = []
            for i in range(N):
                imageDataset = imageDataset_list[i]
                output = conv_segment(imageDataset.images)
                output_list.append(output)
            losed_output_list = lose_something(output_list, loss_num)
            decoded_output_list = decoder(losed_output_list)
            output = torch.cat(decoded_output_list, dim=3)
            output = output.view(output.size(0), -1)

            _, predicted = torch.max(fc_segment(output).data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"样本总数: {total}")
    print(
        f"原始模型(model) -> 预测正确数: {original_correct}, 预测准确率: {100 * original_correct / total}%"
    )
    print(
        f"原始模型(conv+fc) -> 预测正确数: {merge_correct}, 预测准确率: {100 * merge_correct / total}%"
    )
    print(
        f"使用Encoder和Decoder -> 预测正确数: {correct}, 预测准确率: {100 * correct / total}%"
    )

In [ ]:
# 训练集
print("训练")
for i in range(N + 1):
    print(f"loss_num: {i}")
    evaluation(train_loader, i)


训练
loss_num: 0


Evaluating...: 100%|████████████████████| 938/938 [00:26<00:00, 34.83it/s]


样本总数: 60000
原始模型(model) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
原始模型(conv+fc) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
使用Encoder和Decoder -> 预测正确数: 41842, 预测准确率: 69.73666666666666%
loss_num: 1


Evaluating...: 100%|████████████████████| 938/938 [00:26<00:00, 35.61it/s]


样本总数: 60000
原始模型(model) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
原始模型(conv+fc) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
使用Encoder和Decoder -> 预测正确数: 39695, 预测准确率: 66.15833333333333%
loss_num: 2


...

样本总数: 60000
原始模型(model) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
原始模型(conv+fc) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
使用Encoder和Decoder -> 预测正确数: 18737, 预测准确率: 31.22833333333333%
loss_num: 5


Evaluating...: 100%|████████████████████| 938/938 [00:25<00:00, 36.12it/s]


样本总数: 60000
原始模型(model) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
原始模型(conv+fc) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
使用Encoder和Decoder -> 预测正确数: 10551, 预测准确率: 17.585%
loss_num: 6


Evaluating...: 100%|████████████████████| 938/938 [00:25<00:00, 36.51it/s]

样本总数: 60000
原始模型(model) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
原始模型(conv+fc) -> 预测正确数: 59612, 预测准确率: 99.35333333333334%
使用Encoder和Decoder -> 预测正确数: 6742, 预测准确率: 11.236666666666666%


In [ ]:
# 测试集
print("测试")
for i in range(N + 1):
    print(f"loss_num: {i}")
    evaluation(test_loader, i)

测试
loss_num: 0


Evaluating...: 100%|████████████████████| 157/157 [00:04<00:00, 35.57it/s]


样本总数: 10000
原始模型(model) -> 预测正确数: 9891, 预测准确率: 98.91%
原始模型(conv+fc) -> 预测正确数: 9891, 预测准确率: 98.91%
使用Encoder和Decoder -> 预测正确数: 7073, 预测准确率: 70.73%
loss_num: 1


Evaluating...: 100%|████████████████████| 157/157 [00:04<00:00, 35.43it/s]


样本总数: 10000
原始模型(model) -> 预测正确数: 9891, 预测准确率: 98.91%
原始模型(conv+fc) -> 预测正确数: 9891, 预测准确率: 98.91%
使用Encoder和Decoder -> 预测正确数: 6741, 预测准确率: 67.41%
loss_num: 2


...

样本总数: 10000
原始模型(model) -> 预测正确数: 9891, 预测准确率: 98.91%
原始模型(conv+fc) -> 预测正确数: 9891, 预测准确率: 98.91%
使用Encoder和Decoder -> 预测正确数: 3272, 预测准确率: 32.72%
loss_num: 5


Evaluating...: 100%|████████████████████| 157/157 [00:04<00:00, 35.60it/s]


样本总数: 10000
原始模型(model) -> 预测正确数: 9891, 预测准确率: 98.91%
原始模型(conv+fc) -> 预测正确数: 9891, 预测准确率: 98.91%
使用Encoder和Decoder -> 预测正确数: 1913, 预测准确率: 19.13%
loss_num: 6


Evaluating...: 100%|████████████████████| 157/157 [00:04<00:00, 34.81it/s]

样本总数: 10000
原始模型(model) -> 预测正确数: 9891, 预测准确率: 98.91%
原始模型(conv+fc) -> 预测正确数: 9891, 预测准确率: 98.91%
使用Encoder和Decoder -> 预测正确数: 1135, 预测准确率: 11.35%
